#Random Forest


## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.

In [14]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data('non-secuencial')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total_for_non_serial.csv
(30555, 33)
________________________________________________________________________________


## Preparación de los datos

In [15]:
x_data = esios_controller.get_data_real_time()
y_data = esios_controller.get_target_data()

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (20471, 23)
Ytrain_dim: (20471, 1)


## Modelo

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regressor = RandomForestRegressor(random_state=0, n_estimators=100)
regressor.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [17]:

cross_val_score(regressor, x_train, y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 

array([0.8810945 , 0.90707287, 0.90307692, 0.87740886, 0.90942654,
       0.92198088, 0.79639903, 0.86748683, 0.92813885, 0.93621009])

In [19]:
print('R2 en entrenamiento es: ', regressor.score(x_train, y_train))
print('R2 en validación es: ', regressor.score(x_valid, y_valid))

print('MAE: ', mean_absolute_error(regressor.predict(x_valid), y_valid))
print('MSE: ', mean_squared_error(regressor.predict(x_valid), y_valid))
#print('RMSE: ', mean_squared_log_error(regressor.predict(x_valid), y_valid))
print('Variance: ', explained_variance_score(regressor.predict(x_valid), y_valid))
print('R2: ', r2_score(regressor.predict(x_valid), y_valid))
#print(regressor.coef_)

R2 en entrenamiento es:  0.9940292400218449
R2 en validación es:  0.9438712631769839
MAE:  1.8953539964299895
MSE:  7.106596770213214
Variance:  0.9385451796252975
R2:  0.9374660891991529


## Normalizados

In [0]:
scaler = StandardScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)
x_valid_est = scaler.fit_transform(x_valid)
y_valid_est = scaler.fit_transform(y_valid)

In [21]:
regressor_standarized = RandomForestRegressor(random_state=0, n_estimators=100)
regressor_standarized.fit(x_train_est, y_train_est)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [22]:
print('R2 en entrenamiento es: ', regressor_standarized.score(x_train_est, y_train_est))
print('R2 en validación es: ', regressor_standarized.score(x_valid_est, y_valid_est))

print('MAE: ', mean_absolute_error(regressor_standarized.predict(x_valid_est), y_valid_est))
print('MSE: ', mean_squared_error(regressor_standarized.predict(x_valid_est), y_valid_est))
#print('RMSE: ', mean_squared_log_error(regressor_standarized.predict(x_valid_est), y_valid_est))
print('Variance: ', explained_variance_score(regressor_standarized.predict(x_valid_est), y_valid_est))
print('R2: ', r2_score(regressor_standarized.predict(x_valid_est), y_valid_est))

R2 en entrenamiento es:  0.9940388820147471
R2 en validación es:  0.9314792572447922
MAE:  0.17759906622482613
MSE:  0.06852074275520786
Variance:  0.9209749632387114
R2:  0.9209499575475414


## Optimización de modelos

In [0]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

clf = GridSearchCV(regressor, param_grid, cv=5)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


## Visualización del modelo

In [0]:
# Extract single tree
estimator = regressor.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = x_valid_est.feature_names,
                class_names = y_valid_est.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

AttributeError: ignored